<a name='1'></a>
## Import Packages and load data

In [1]:
### -*- coding: utf-8 -*-
"""
Created on Mon May 24 13:26:13 2021

@author: kjsanche

Description: 
A function to load the 5 minute granules from MODIS channel 1 
(0.65 microns) and the contrail mask for ML with a CNN.

To do:

-organize/markdown/comment code


Input:
Path   (string)
 
        
        
Output:
MODISCh1 (2D uint32)
MASK     (2D uint16)
"""


from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib as mpl
from itertools import compress
import numpy as np
import struct
import os
import glob
import random
from twilio.rest import Client
from decouple import config
from UNET_Functions import unet_model, summary
from Sat_contrail_read import Extract_RawDef, extract_img, extract_mask, extract_imglist, get_model_memory_usage
import tensorflow as tf
import sys
sys.path.append('/home/kjsanche/Desktop/Projects/loss')
from loss_function import *
from tensorflow.python.ops.metrics_impl import false_positives, false_negatives
import tensorflow.keras.metrics as tfm
import tensorflow_addons as tfa
from focal_loss import BinaryFocalLoss, SparseCategoricalFocalLoss


#config = tf.config.experimental
#config.gpu_options.allow_growth = True
#sess = tf.Session(config=config)

sys_details = tf.sysconfig.get_build_info()
print(sys_details)
cudnn_version = sys_details["cudnn_version"]
cuda_version = sys_details["cuda_version"]

print('cuda version: ', cuda_version)
print('cudNN version: ',cudnn_version)
print('TF version: ', tf.version.VERSION)




VALIDATION_SPLIT = .2
BATCH_SIZE = 64
EPOCHS = 500

IMG_W=64
IMG_H=64
N_CHANNELS = 3
N_FILTERS = 64
LEARNING_RATE = 0.00003
AUTO = tf.data.AUTOTUNE # used in tf.data.Dataset API
VERSION = '08'
TFrecord_path ='/home/kjsanche/Desktop/ExternalSSD/SatContrailData/TFrecords/'
Models_path ='/home/kjsanche/Desktop/ExternalSSD/SatContrailData/Models/'

training_filenames=sorted(tf.io.gfile.glob([TFrecord_path + '*v' + VERSION + '.tfrecords']))

if N_CHANNELS == 3:
    validation_filenames=sorted(tf.io.gfile.glob([TFrecord_path + '*v3channelVALIDATION.tfrecords']))
elif N_CHANNELS == 7:
    validation_filenames=sorted(tf.io.gfile.glob([TFrecord_path + '*vVALIDATION.tfrecords']))


random.Random(5).shuffle(training_filenames)
random.Random(5).shuffle(validation_filenames)

print(len(training_filenames))
print(len(validation_filenames))

OrderedDict([('cpu_compiler', '/home/builder/ktietz/aggregate/tensorflow_recipes/ci_cpu/tensorflow-base_1614583966145/_build_env/bin/x86_64-conda_cos6-linux-gnu-gcc'), ('cuda_compute_capabilities', ['compute_35', 'compute_52', 'compute_60', 'compute_61', 'compute_70', 'compute_75']), ('cuda_version', '10.1'), ('cudnn_version', '7'), ('is_cuda_build', True), ('is_rocm_build', False)])
cuda version:  10.1
cudNN version:  7
TF version:  2.4.1
40
20


In [2]:
account_sid = config('ACCOUNT_ID',default='')
auth_token = config('AUTHENTICATION_TOKEN',default='')
phone_num = config('PHONE_NUMBER',default='')
client = Client(account_sid, auth_token)

#message = client.messages .create(
#                    body =  "Testing 1, 2, 3, 4", #Message you send
#                    from_ = '+12184383951',#Provided phone number
#                    to =    phone_num)#Your phone number
#message.sid

class SMSCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if epoch%50 == 0:
            FP = logs['val_false_positives']
            TP = logs['val_true_positives']
            FN = logs['val_false_negatives']
            TN = logs['val_true_negatives']
            IOU_val = TP/(FN+TP+FP)
            message = client.messages .create(
                
                body = "For Epoch:{}, "
                        "Training loss={:.2f}, Validation loss ={:.2f} and Validation IOU = {:.2f}"
                .format(epoch, logs["loss"], logs["val_loss"], IOU_val),

                from_ = "+12184383951", to = phone_num)

            print(message.sid)

In [3]:
def parse_tfr_element(element):
    
    data = {
      'height': tf.io.FixedLenFeature([], tf.int64),
      'width':tf.io.FixedLenFeature([], tf.int64),
      'depth':tf.io.FixedLenFeature([], tf.int64),
      'raw_label':tf.io.FixedLenFeature([], tf.string),#tf.string = bytestring (not text string)
      'raw_image' : tf.io.FixedLenFeature([], tf.string),#tf.string = bytestring (not text string)
    }


    content = tf.io.parse_single_example(element, data)

    height = content['height']
    width = content['width']
    depth = content['depth']
    raw_label = content['raw_label']
    raw_image = content['raw_image']


    #get our 'feature'-- our image -- and reshape it appropriately
    feature = tf.io.parse_tensor(raw_image, out_type=tf.float16)
    feature = tf.reshape(feature, shape=[height,width,depth])
    label = tf.io.parse_tensor(raw_label, out_type=tf.int8)
    label = tf.reshape(label, shape=[height,width])
    return (feature, label)

def get_batched_dataset(filenames, testing = False):
    option_no_order = tf.data.Options()
    if testing:
        option_no_order.experimental_deterministic = True
    else:
        option_no_order.experimental_deterministic = False

    #dataset = tf.data.TFRecordDataset(filenames)
    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.with_options(option_no_order)
    dataset = dataset.interleave(tf.data.TFRecordDataset, num_parallel_calls=AUTO)
    dataset = dataset.map(parse_tfr_element, num_parallel_calls=AUTO)

    dataset = dataset.cache() # If dataset fits in RAM
    if not testing:
        dataset = dataset.shuffle(2048)
    #dataset = dataset.repeat()
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True) 
    dataset = dataset.prefetch(AUTO) #

    return dataset


def get_training_dataset(training_filenames):
    return get_batched_dataset(training_filenames)

def get_validation_dataset(validation_filenames):
    return get_batched_dataset(validation_filenames)

def get_test_dataset(test_filenames):
    return get_batched_dataset(test_filenames, testing = True)

def display(display_list):
    plt.figure(figsize=(15, 15))

    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        #print(i)
        #print(display_list[i].shape)
        if i == 0:
            plt.imshow(np.float32(display_list[i][:,:,7]))#-display_list[i][:,:,1]))
        else:
            plt.imshow(np.float32(1*display_list[i]))
        plt.axis('off')
    plt.show()

The below code cell uses a lot of memory and therefore should only be used for testing and not be used during training.

In [4]:
def FocalTverskyLoss(targets, inputs, alpha=0.5, beta=0.5, gamma=1, smooth=1e-6):
        '''
        ... in the case of α=β=0.5 the Tversky index simplifies to be 
        the same as the Dice coefficient, which is also equal to the F1 
        score. With α=β=1, Equation 2 produces Tanimoto coefficient, and 
        setting α+β=1 produces the set of Fβ scores. Larger βs weigh 
        recall higher than precision (by placing more emphasis on false negatives).
        '''
        targets = tf.cast(targets,tf.float32)
        #flatten label and prediction tensors
        inputs = K.flatten(inputs)
        targets = K.flatten(targets)
        
        #True Positives, False Positives & False Negatives
        TP = K.sum((inputs * targets))
        FP = K.sum(((1-targets) * inputs))
        FN = K.sum((targets * (1-inputs)))
               
        Tversky = (TP + smooth) / (TP + alpha*FP + beta*FN + smooth) 

        softdice = 2*TP/(K.sum(inputs**2)+K.sum(targets**2)+smooth)
        Tversky = softdice
        
        FocalTversky = K.pow((1 - Tversky), gamma)

        
        
        return FocalTversky

In [5]:

#gamma>1 reduces the relative loss for well-classified examples 
#alpha is a weighted term whose value is α for positive(foreground) alpha = 1 does nothing. alpha = 0.25 is best
#class and 1-α for negative(background) class.

unet = unet_model((IMG_W, IMG_H, N_CHANNELS),n_filters=N_FILTERS,n_classes=1)
#loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#loss=tfa.losses.SigmoidFocalCrossEntropy(),
#loss=[BinaryFocalLoss(gamma=2,from_logits=True)],

#Larger βs weigh recall higher than precision (by placing more emphasis on false negatives)
#loss=FocalTverskyLoss(targets, inputs, alpha=0.5, beta=0.5, gamma=1, smooth=1e-6)
unet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss=[FocalTverskyLoss],
              metrics=[tfm.Precision(), tfm.Recall(), tfm.FalseNegatives(), tfm.FalsePositives(), tfm.TruePositives(), tfm.TrueNegatives()])

unet.summary(line_length = 130)
print('mem usage: ', tf.config.experimental.get_memory_usage("GPU:0"))

Model: "model"
__________________________________________________________________________________________________________________________________
Layer (type)                              Output Shape                 Param #         Connected to                               
input_1 (InputLayer)                      [(None, 64, 64, 3)]          0                                                          
__________________________________________________________________________________________________________________________________
conv2d (Conv2D)                           (None, 64, 64, 64)           1792            input_1[0][0]                              
__________________________________________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)                   (None, 64, 64, 64)           0               conv2d[0][0]                               
____________________________________________________________________

In [ ]:
print(get_model_memory_usage(BATCH_SIZE, unet))

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta = 0.005, patience=20, mode ="min", verbose = 2, restore_best_weights=True)
training_data = get_training_dataset(training_filenames)
validation_data = get_validation_dataset(validation_filenames)
#test_data = get_test_dataset(test_filenames)
print(training_data)
model_history = unet.fit(training_data, validation_data=validation_data, epochs=EPOCHS, callbacks = [SMSCallback(), callback])



1.425
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
<PrefetchDataset shapes: ((64, None, None, None), (64, None, None)), types: (tf.float16, tf.int8)>
Epoch 1/500
378/378 [==============================] - 239s 535ms/step - loss: 0.8600 - precision: 0.0099 - recall: 0.0056 - false_negatives: 814620.5488 - false_positives: 154404.0712 - true_positives: 1566.7916 - true_negatives: 48836072.6781 - val_loss: 0.9964 - val_precision: 8.7203e-04 - v

## unet.save_weights(Models_path+'model')
#unet = tf.keras.models.load_model(Models_path+'model')

In [ ]:

fig, axs = plt.subplots(2,2, figsize=(15,15))

axs[0,0].plot(model_history.history['loss'])
axs[0,0].plot(model_history.history['val_loss'])
axs[0,0].set_title('model loss')
axs[0,0].set_ylabel('loss')

#axs[0].legend(['train', 'val'], loc='upper left')
#axs[0].show()





F1 = 2*np.divide(np.multiply(model_history.history['precision'],model_history.history['recall']), np.add(model_history.history['precision'], model_history.history['recall']))
F1_val = 2*np.divide(np.multiply(model_history.history['val_precision'],model_history.history['val_recall']), np.add(model_history.history['val_precision'], model_history.history['val_recall']))
FP = model_history.history['false_positives']
TP = model_history.history['true_positives']
FN = model_history.history['false_negatives']
TN = model_history.history['true_negatives']
IOU = np.divide(TP,np.add(FN,np.add(TP,FP)))
MCC = np.subtract(np.multiply(TP,TN),np.multiply(FP,FN))/np.sqrt(np.multiply(np.multiply(np.add(TP,FP),np.add(TP,FN)), np.multiply(np.add(TN,FP),np.add(TN,FN))))
FP = model_history.history['val_false_positives']
TP = model_history.history['val_true_positives']
FN = model_history.history['val_false_negatives']
TN = model_history.history['val_true_negatives']
MCC_val = np.subtract(np.multiply(TP,TN),np.multiply(FP,FN))/np.sqrt(np.multiply(np.multiply(np.add(TP,FP),np.add(TP,FN)), np.multiply(np.add(TN,FP),np.add(TN,FN))))
IOU_val = np.divide(TP,np.add(FN,np.add(TP,FP)))

axs[1,0].plot(IOU)
axs[1,0].plot(IOU_val)
axs[1,0].set_title(f'model IoU, max = {np.max(IOU_val)}')
axs[1,0].set_ylabel('IoU')
axs[1,0].legend(['train', 'val'], loc='upper right')

axs[0,1].plot(F1)
axs[0,1].plot(F1_val)
axs[0,1].set_title('model F1 score')
axs[0,1].set_ylabel('F1')
axs[0,1].set_xlabel('epoch')

axs[1,1].plot(MCC)
axs[1,1].plot(MCC_val)
axs[1,1].set_title('model MCC')
axs[1,1].set_ylabel('MCC')
axs[1,1].set_xlabel('epoch')


i = 1
while os.path.exists(Models_path+"Model%s.png" % i):
    i += 1
plt.savefig(Models_path+"Model%s.png" % i)

# Save the weights 
unet.save_weights(Models_path+ "Model%s.ckpt" % i)

In [ ]:
 message = client.messages .create(
                
    body = "Model stopped at Epoch:{}, "
            "max Validation IOU = {:.2f}"
    .format(len(IOU_val), np.max(IOU_val)),

    from_ = "+12184383951", to = phone_num)

print(message.sid)

In [ ]:
def blockshaped(arr, nrows, ncols):
    """
    Return an array of shape (n, nrows, ncols) where
    n * nrows * ncols = arr.size

    If arr is a 2D array, the returned array should look like n subblocks with
    each subblock preserving the "physical" layout of arr.
    """
    h, w = arr.shape
    assert h % nrows == 0, f"{h} rows is not evenly divisible by {nrows}"
    assert w % ncols == 0, f"{w} cols is not evenly divisible by {ncols}"
    return (arr.reshape(h//nrows, nrows, -1, ncols)
               .swapaxes(1,2)
               .reshape(-1, nrows, ncols))
def unblockshaped(arr, h, w):
    """
    Return an array of shape (h, w) where
    h * w = arr.size

    If arr is of shape (n, nrows, ncols), n sublocks of shape (nrows, ncols),
    then the returned array preserves the "physical" layout of the sublocks.
    """
    n, nrows, ncols = arr.shape
    return (arr.reshape(h//nrows, -1, nrows, ncols)
               .swapaxes(1,2)
               .reshape(h, w))

In [ ]:
def image_IOU(mask1, mask2):
    inter = 0
    union = 0
    #for _, mask in image_ds:
    #mask = mask.numpy()
    mask1[mask1>1]=1
    mask2[mask2>1]=1
    mask1[mask1<1]=0
    mask2[mask2<1]=0

    #create list of true indicies for both masks
    a = list(compress(range(len(mask1[:,:].flatten())), mask1[:,:].flatten()))
    b = list(compress(range(len(mask2[:,:].flatten())), mask2[:,:].flatten()))

    IOU = len(list(set(a).intersection(b)))/len(list(set(a).union(b)))
    inter += len(list(set(a).intersection(b)))
    union += len(list(set(a).union(b)))
    #print(f"intersection = {len(list(set(a).intersection(b)))}")
    #print(f"union = {len(list(set(a).union(b)))}")
    print(f"IOU ={IOU}")
    return inter, union, IOU

In [ ]:

def rescale_image(img):
    #if NCHANNELS-5>0:
    NCHANNELS = 7
    tmp = img[:,:,:,NCHANNELS-5:]
    tmp[tmp==17000]=0
    img[:,:,:,NCHANNELS-5:]=tmp
    BT_rng = np.float64([17000, 34000]) #this range is good for 4 upper channels. assuming range of 170-340 K (BT is multiplied by 100)
    img[:,:,:,NCHANNELS-5:-1] = (img[:,:,:,NCHANNELS-5:-1] * ((BT_rng[1]-BT_rng[0])/2) ) + np.mean(BT_rng)
    BT_rng = np.float64([0, 66535]) #lower 3 channels have a much larger range.
    img[:,:,:,:NCHANNELS-5] = (img[:,:,:,:NCHANNELS-5] * ((BT_rng[1]-BT_rng[0])/2) ) + np.mean(BT_rng)
    return img
    #else:
    #   img_rotated[img_rotated==0]=17000
    #   BT_rng = np.float64([17000, 34000]) #this range is good for 4 upper channels. assuming range of 170-340 K (BT is multiplied by 100)
    #   img_rotated[:,:,:,:-1] = (img_rotated[:,:,:,:-1]-np.mean(BT_rng)) / ((BT_rng[1]-BT_rng[0])/2) 

In [ ]:
def create_mask(pred_mask):
    pred_mask = tf.argmax(pred_mask, axis=-1)
    #print(pred_mask.shape)
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask[0]

def display2(display_list):
    plt.figure(figsize=(15, 15))

    title = ['Input Image', 'Old Mask', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        #print(i)
        #print(display_list[i].shape)
        if i == 0:
            plt.imshow(np.float32(display_list[i][:,:,6]-display_list[i][:,:,5]), cmap = 'gray')
        #elif i == 1:
        #    plt.imshow(np.float32(display_list[i][:,:,0]-display_list[i][:,:,1]), cmap = 'gray')
        elif i < 3:
            plt.imshow(np.float32(1*display_list[i].numpy().squeeze()), cmap = 'gray')
        else:
            plt.imshow(np.float32(1*display_list[i].squeeze()), cmap = 'gray')
        plt.axis('off')
    _ = unet.evaluate(display_list[0][tf.newaxis,:,:,:], display_list[2][tf.newaxis,:,:], verbose=2)
    #print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))
    plt.show()

def show_predictions(dataset=None,num=1):
    """
    Displays the first image of each of the num batches
    """
    if dataset:
        for test_data, oldMask_data in dataset.take(num):
            image, mask = test_data
            image2, old_mask = oldMask_data
            

            _, _, _, FN, FP, TP, TN, _ = unet.evaluate(image.numpy(), mask.numpy(), verbose=2)
            IOU = np.divide(TP,np.add(FN,np.add(TP,FP)))
            print(f'IOU = {IOU}')
            
            #rescale image (scaled previously for training)
            origImage = rescale_image(np.array(image,dtype=float))


            #compile full image
            fullimage = np.empty([2048, 4096, 7])
            for i in range(origImage.shape[-1]):
                fullimage[:,:,i] = unblockshaped(origImage[:,:,:,i], 2048, 4096)

       

            #compile both masks
            fullOldMask = unblockshaped(old_mask.numpy(), 2048, 4096)
            fullNewMask = unblockshaped(mask.numpy(), 2048, 4096)
            
            #plot image and masks
            fig = plt.figure()
            plt.imshow(fullimage[:,:,4]-fullimage[:,:,5], cmap = 'gray')
            plt.show()
            fig = plt.figure()
            plt.imshow(np.float32(fullOldMask), cmap = 'gray')
            plt.show()
            Oldinter, Oldunion, OldIOU = image_IOU(np.float32(fullOldMask), np.float32(fullNewMask))
            fig = plt.figure()
            plt.imshow(np.float32(fullNewMask), cmap = 'gray')
            plt.show()

            #run model for case to produced predicted mask and plot
            pred_mask = unet.predict(image)
            fullPredMask = unblockshaped(pred_mask, 2048, 4096)
            Pinter, Punion, PIOU = image_IOU(np.float32(fullNewMask), fullPredMask)
            fig = plt.figure()
            plt.imshow(np.float32(fullPredMask), cmap = 'gray')
            plt.show()
     
            

    else:
        display([sample_image, sample_mask,
             create_mask(unet.predict(sample_image[tf.newaxis, ...]))])


In [ ]:
def IOU_test(dataset=None,num=1):
    """
    extract IOU values for each image
    """

    for test_data, oldMask_data in dataset.take(num):
        image, mask = test_data
        image2, old_mask = oldMask_data


        #_, _, _, FN, FP, TP, TN, _ = unet.evaluate(image.numpy(), mask.numpy(), verbose=2)
        #IOU = np.divide(TP,np.add(FN,np.add(TP,FP)))
        #print(f'IOU = {IOU}')

        #rescale image (scaled previously for training)
        #origImage = rescale_image(np.array(image,dtype=float))


        #compile full image
        #fullimage = np.empty([2048, 4096, 7])
        #for i in range(origImage.shape[-1]):
        #    fullimage[:,:,i] = unblockshaped(origImage[:,:,:,i], 2048, 4096)



        #compile both masks
        fullOldMask = unblockshaped(old_mask.numpy(), 2048, 4096)
        fullNewMask = unblockshaped(mask.numpy(), 2048, 4096)

        #plot image and masks
        #fig = plt.figure()
        #plt.imshow(fullimage[:,:,4]-fullimage[:,:,5], cmap = 'gray')
        #plt.show()
        #fig = plt.figure()
        #plt.imshow(np.float32(fullOldMask), cmap = 'gray')
        #plt.show()
        Oldinter, Oldunion, OldIOU = image_IOU(np.float32(fullOldMask), np.float32(fullNewMask))
        #fig = plt.figure()
        #plt.imshow(np.float32(fullNewMask), cmap = 'gray')
        #plt.show()

        #run model for case to produced predicted mask and plot
        pred_mask = unet.predict(image)
        fullPredMask = unblockshaped(pred_mask, 2048, 4096)
        Pinter, Punion, PIOU = image_IOU(np.float32(fullNewMask), fullPredMask)
        #fig = plt.figure()
        #plt.imshow(np.float32(fullPredMask), cmap = 'gray')
        #plt.show()
    return Oldinter, Oldunion, OldIOU, Pinter, Punion, PIOU


In [ ]:
import matplotlib as mpl
from itertools import compress
mpl.rcParams['figure.dpi']= 1000
print(len(test_filenames))
Oldinter = np.empty([len(test_filenames)])
Oldunion = np.empty([len(test_filenames)])
OldIOU = np.empty([len(test_filenames)])
Pinter = np.empty([len(test_filenames)])
Punion = np.empty([len(test_filenames)])
PIOU = np.empty([len(test_filenames)])
for cnt, (fname, fnameoldMask) in enumerate(zip(test_filenames,test_OldMASKfilenames)): 
    test_data = get_test_dataset(fname)
    oldMask_data = get_test_dataset(fnameoldMask)
    combined_dataset = tf.data.Dataset.zip((test_data,oldMask_data))
    #show_predictions(combined_dataset, 1)
    Oldinter[cnt], Oldunion[cnt], OldIOU[cnt], Pinter[cnt], Punion[cnt], PIOU[cnt] = IOU_test(combined_dataset, 1)